# augmentation
* Efficient-net B2
* Augmentation
    - Transpose # 행렬 스왑
    - HorizontalFlip # 좌우 반전
    - VerticalFlip # 상하 반전
    - ShiftScaleRotate # 랜덤하게 옮기고, scale, 회전
    - HueSaturationValue # 빛깔, 색조, 값 변환
    - RandomBrightnessContrast # 명도 대비
    - ChannelShuffle # RGB 채널 간 shuffle
    - CoarseDropout # 조그마한 검은 사각형 영역 추가
* lr scheduler
    - Custom CosineAnnealingWarmUpRestarts
* k-fold
* result
    - train loss : 
    - validation loss : 
    - f1 score : 
    - epoch : 

In [1]:
import gc
import os
import random
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import timm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [36]:
CFG = {
    'IMG_SIZE': 260,
    'EPOCHS': 1000,
    'LEARNING_RATE': 1e-6,
    'BATCH_SIZE': 16,
    'PATIENCE': 10,
    'WARMUP': 5,
    'K-FOLD': 5,
    'FILENAME': 'kfold',
    'SEED': 6
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
running_colab = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
if running_colab:
    from google.colab import drive
    drive.mount('/content/drive')
if running_colab:
    data_path = '/content/drive/MyDrive/Colab Notebooks/ai6th/data/optiver/'
else:
    data_path = '../../data/'

In [6]:
df = pd.read_csv(os.path.join(data_path, 'train.csv'))
df.loc[3896, 'artist'] = 'Titian'
df.loc[3986, 'artist'] = 'Alfred Sisley'

In [7]:
artists = df.groupby('artist')[['id']].count().rename(columns={'id':'count'}).reset_index()

In [8]:
# Label Encoding
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)

In [9]:
# train_df, val_df = train_test_split(df, test_size=0.2, random_state=CFG['SEED'])

In [10]:
# train_df = train_df.sort_values(by=['id'])
# train_df.head()

In [11]:
# val_df = val_df.sort_values(by=['id'])
# val_df.head()

In [12]:
def get_data(df, infer=False):
    if infer:
        return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values
    return df['img_path'].apply(lambda p: os.path.join(data_path, p)).values, df['artist'].values

In [13]:
from torchvision.transforms import ToTensor


class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms if transforms else ToTensor()

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_paths)

In [14]:
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Transpose(p=0.5), # 행렬 스왑
    A.HorizontalFlip(p=0.5), # 좌우 반전
    A.VerticalFlip(p=0.5), # 상하 반전
    A.ShiftScaleRotate(p=0.5), # 랜덤하게 옮기고, scale, 회전
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5), # 빛깔, 색조, 값 변환
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5), # 명도 대비
    A.ChannelShuffle(), # RGB 채널 간 shuffle
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    A.CoarseDropout(p=0.5),
    ToTensorV2()
])

validation_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE']*2,CFG['IMG_SIZE']*2),
    A.RandomCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [15]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

In [16]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('efficientnet_b2', pretrained=True, num_classes=512)
        self.classifier = nn.Sequential(
            nn.LeakyReLU(),
            nn.Linear(512, num_classes)
        )
            
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [17]:
def clear_mem():
    gc.collect()
    torch.cuda.empty_cache()

In [18]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            print(f'Best F1 score from now: {self.best_score}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
        
        return self.early_stop

In [19]:
import math
from torch.optim.lr_scheduler import LRScheduler

class CosineAnnealingWarmUpRestarts(LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [20]:
time_now = datetime.now()
run_id = 0# time_now.strftime("%Y%m%d%H%M%S")
os.makedirs(os.path.join(data_path, f'./runs/{run_id}'), exist_ok=True)
print(f'{run_id=}')

run_id=0


In [21]:
def train(epoch, model, optimizer, criterion, train_loader, device, lr_scheduler=None):
    model.train()
    train_loss = []
    lr_list = []
    # bar = tqdm(enumerate(train_loader), total = len(train_loader), desc='Train Loop')
    for idx, (img, label) in enumerate(train_loader):
        img, label = img.float().to(device), label.long().to(device)
        
        optimizer.zero_grad()

        model_pred = model(img)
        loss = criterion(model_pred, label)

        loss.backward()
        if (epoch*idx)%10 == 0:
            lr_list.append(optimizer.param_groups[0]['lr'])
        optimizer.step()
        train_loss.append(loss.item())
        # bar.set_postfix(train_loss = f'{loss.item():.4f}', lr = f"{optimizer.param_groups[0]['lr']:.6f}")
        if lr_scheduler:
            lr_scheduler.step()
    return np.mean(train_loss), lr_list

In [22]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for img, label in iter(test_loader):
            img, label = img.float().to(device), label.long().to(device)
            
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [23]:
def train_epoch(k, model, optimizer, train_loader, test_loader, lr_scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    early_stopping = EarlyStopping(patience=CFG['PATIENCE'], verbose=True)
    
    best_score = 0
    lr_list = []
    train_loss_list, val_loss_list = [], []
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        tr_loss, lr_ = train(epoch, model, optimizer, criterion, train_loader, device, lr_scheduler)
        val_loss, val_score = validation(model, criterion, test_loader, device)
        train_loss_list.append(tr_loss)
        val_loss_list.append(val_loss)
        
        if lr_scheduler is not None:
            lr_list.extend(lr_)
            
        if best_score < val_score:
            print(f'**Epoch [{k}-{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
            best_score = val_score
            torch.save(model, os.path.join(data_path, f'runs/{run_id}/best_model_{k}.pt'))
        else:
            print(f'Epoch [{k}-{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        clear_mem()
        if early_stopping(val_score):
            print(f'Epoch [{k}-{epoch}], early stopping')
            break
    if lr_list:
        return (train_loss_list, val_loss_list, val_score, lr_list)
    else:
        return (train_loss_list, val_loss_list, val_score, None)

In [24]:
def k_fold(k): # k-fold
    skf = StratifiedKFold(n_splits=k, shuffle=False)
    f1_sum = 0
    for k_, (train_index, valid_index) in enumerate(skf.split(df, df['artist'])):
        print(f'{k_}-fold start')
        
        train_img_paths, train_labels = get_data(df.iloc[train_index])
        val_img_paths, val_labels = get_data(df.iloc[valid_index])
        train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
        train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=0)
        val_dataset = CustomDataset(val_img_paths, val_labels, validation_transform)
        val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=0)
        
        model = BaseModel()
        model.eval()
        optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
        # lr : 5epochs 동안 0.01->0
        lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=CFG['WARMUP']*len(train_loader), T_mult=1, eta_max=0.001, T_up=50, gamma=0.7)
        _, _, f1_score, _ = train_epoch(k_, model, optimizer, train_loader, val_loader, lr_scheduler, device)
        f1_sum += f1_score
    return f1_sum/k

In [25]:
f1_score = k_fold(CFG['K-FOLD'])
print(f1_score)

0-fold start
**Epoch [0-1], Train Loss : [2.75373] Val Loss : [2.12732] Val F1 Score : [0.26879]
**Epoch [0-2], Train Loss : [1.82444] Val Loss : [1.49718] Val F1 Score : [0.47585]
**Epoch [0-3], Train Loss : [1.21246] Val Loss : [1.13499] Val F1 Score : [0.59977]
**Epoch [0-4], Train Loss : [0.71235] Val Loss : [0.97519] Val F1 Score : [0.65915]
**Epoch [0-5], Train Loss : [0.44958] Val Loss : [0.92956] Val F1 Score : [0.68156]
Epoch [0-6], Train Loss : [0.96023] Val Loss : [1.37924] Val F1 Score : [0.54401]
EarlyStopping counter: 1 out of 10
Best F1 score from now: 0.6815576731640702
Epoch [0-7], Train Loss : [0.76667] Val Loss : [1.14516] Val F1 Score : [0.63448]
EarlyStopping counter: 2 out of 10
Best F1 score from now: 0.6815576731640702
**Epoch [0-8], Train Loss : [0.42891] Val Loss : [1.00605] Val F1 Score : [0.68348]
**Epoch [0-9], Train Loss : [0.20535] Val Loss : [0.96288] Val F1 Score : [0.70614]
**Epoch [0-10], Train Loss : [0.11450] Val Loss : [0.95509] Val F1 Score : [0.7

In [26]:
test_df = pd.read_csv(os.path.join(data_path, './test.csv'))
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [27]:
test_img_paths = get_data(test_df, infer=True)

In [28]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [29]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for idx, img in enumerate(test_loader):
            img = img.float().to(device)
            
            model_pred = model(img).detach().cpu()
            model_pred = F.softmax(model_pred, dim=1)
            model_preds.extend(model_pred.numpy().tolist())
    
    print('Done.')
    return model_preds

In [37]:
CFG['K-FOLD']

5

In [39]:
result_df = pd.DataFrame(np.zeros((test_df.shape[0], len(le.classes_))))
for k_ in range(CFG['K-FOLD']):
    checkpoint = os.path.join(data_path, f'runs/{run_id}/best_model_{k_}.pt')
    print(f'{k_}-fold CHECKPOINT LOADED: {checkpoint}')
    infer_model = torch.load(checkpoint)
    infer_model.to(device)
    infer_model.eval()
    result_df += pd.DataFrame(inference(infer_model, test_loader, device))

0-fold CHECKPOINT LOADED: ../../data/runs/0/best_model_0.pt
Done.
1-fold CHECKPOINT LOADED: ../../data/runs/0/best_model_1.pt
Done.
2-fold CHECKPOINT LOADED: ../../data/runs/0/best_model_2.pt
Done.
3-fold CHECKPOINT LOADED: ../../data/runs/0/best_model_3.pt
Done.
4-fold CHECKPOINT LOADED: ../../data/runs/0/best_model_4.pt
Done.


In [40]:
result_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-62.470020,-57.348465,-28.817992,-62.817368,-57.471887,-36.364818,-66.104985,-37.131560,-60.939992,-22.376877,...,-35.448220,-72.681121,-37.305878,-20.266613,-52.030369,-43.977599,-23.647032,-57.112928,0.056618,-61.937872
1,-64.075653,-48.206960,12.316254,-47.369438,-59.318394,-47.018031,-110.000351,-23.122041,-37.767885,-42.039499,...,-42.481158,-81.429304,-63.540862,-33.908293,-79.881509,-59.714723,-46.505914,-60.559261,7.565762,-57.299839
2,-24.808428,-76.720341,-65.248060,-69.722178,-78.021451,-86.702891,-20.619292,-78.038012,-59.639781,-12.079488,...,-40.927609,-42.537519,-30.861104,-30.281724,-19.960790,-38.446317,4.235178,-92.654685,-32.724604,-66.194557
3,42.859994,-40.027626,-60.699536,-44.889693,-50.669916,-67.121737,-71.479465,-44.793132,-53.289897,-71.476461,...,-7.869373,-65.126678,-36.349402,-39.498280,-38.572878,-59.836757,-38.941210,-66.893264,11.622741,-35.536864
4,-70.417142,-41.961115,-9.616668,-36.212737,-43.427022,-47.485895,-116.570011,-23.878116,-36.819629,-56.894391,...,-48.085875,-94.639240,-83.537552,-36.040559,-73.515499,-56.719199,-71.430058,-47.254330,16.521688,-58.858765


In [41]:
preds = result_df.idxmax(axis=1)

In [42]:
preds = le.inverse_transform(preds)

In [43]:
submit = pd.read_csv(os.path.join(data_path, './sample_submission.csv'))

In [44]:
submit['artist'] = preds

In [45]:
submit.head()

,id,artist
0,TEST_00000,Edgar Degas
1,TEST_00001,Amedeo Modigliani
2,TEST_00002,Titian
3,TEST_00003,Albrecht Du rer
4,TEST_00004,Vincent van Gogh


In [46]:
submit.to_csv(os.path.join(data_path, f"./submit_{CFG['FILENAME']}.csv"), index=False)